# Vision-Based Navigation with Memory — Full Demo

**FINAL INTEGRATED DEMO**

This notebook demonstrates the complete camera-only navigation system with:
- Optical flow perception (Mode A)
- Temporal memory and smoothing
- Finite state machine with anti-oscillation
- Real-time overlay visualization
- Performance metrics

**To run**: Restart Kernel → Run All

## Setup and Configuration

In [ ]:
# Imports
from pathlib import Path
import sys
import time

import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Project modules
from src.utils import (
    load_config,
    ensure_demo_video,
    open_capture,
    iter_frames,
    release_capture,
    draw_navigation_overlay,
)
from src.perception_flow import compute_optical_flow, compute_obstacle_risk, compute_openness
from src.memory import NavigationMemory
from src.fsm import NavigationFSM
from src.controller import NavigationController
from src.metrics import NavigationMetrics

print("✓ All modules imported successfully")

In [ ]:
# Load configuration
config = load_config("configs/default.yaml")
print("Configuration loaded:")
print(f"  Optical flow method: {config['flow']['method']}")
print(f"  Risk thresholds: avoid={config['risk']['thresholds']['avoid']}, stop={config['risk']['thresholds']['stop']}")
print(f"  FSM default state: {config['fsm']['default_state']}")
print(f"  Recovery frames: {config['memory']['recovery_frames']}")

In [ ]:
# Ensure demo video exists (auto-download if missing)
video_path = ensure_demo_video(config)
print(f"✓ Video ready: {video_path.resolve()}")
print(f"  Exists: {video_path.exists()}")

## Initialize Navigation System

In [ ]:
# Initialize all components
memory = NavigationMemory(config)
fsm = NavigationFSM(config)
controller = NavigationController(config)
metrics = NavigationMetrics()

print("✓ Navigation system initialized:")
print(f"  Memory: risk_window={config['risk']['risk_smooth_window']}, cooldown={config['memory']['cooldown_frames']}")
print(f"  FSM: {fsm.default_state} → transitions based on risk/openness")
print(f"  Controller: linear_v={controller.linear_v}, angular_w={controller.angular_w}")
print(f"  Metrics: tracking FPS, latency, and state durations")

## Run Full Navigation Loop

In [ ]:
# Open video capture
cap = open_capture(config)
max_frames = config["video"].get("max_frames") or 300  # Process up to 300 frames

# Start metrics
metrics.start()

# Storage for visualization
overlay_frames = []
frame_indices = []
sample_interval = 25  # Sample every N frames for display

prev_gray = None
process_count = 0

print(f"Processing up to {max_frames} frames...")
print("Pipeline: Optical Flow → Risk/Openness → Memory → FSM → Controller → Overlay")
print()

for idx, frame in tqdm(iter_frames(cap, max_frames=max_frames, convert_rgb=False), total=max_frames, desc="Processing"):
    frame_start = time.perf_counter()
    
    # Convert to grayscale for optical flow
    curr_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    if prev_gray is not None:
        # PERCEPTION: Compute optical flow and extract signals
        flow = compute_optical_flow(prev_gray, curr_gray, config)
        risk_raw = compute_obstacle_risk(flow, config)
        openness_raw, direction = compute_openness(flow, config)
        
        # MEMORY: Apply temporal smoothing and hysteresis
        signals = memory.update(risk_raw, openness_raw, direction)
        
        # FSM: Determine navigation state
        state = fsm.update(signals)
        
        # CONTROLLER: Generate motion command
        recovery_dir = fsm.get_recovery_direction()
        preferred_dir = recovery_dir if recovery_dir else signals["preferred_direction"]
        command = controller.get_command(state, preferred_dir)
        
        # Compute frame processing latency
        frame_end = time.perf_counter()
        latency = frame_end - frame_start
        
        # METRICS: Record performance
        metrics.record(state, latency)
        
        # VISUALIZATION: Draw overlay
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        fps = metrics.perf.get_fps()
        frame_overlay = draw_navigation_overlay(
            frame_rgb,
            state=state,
            command_name=command.command_name,
            risk=signals["risk_smooth"],
            openness=signals["openness_smooth"],
            fps=fps,
            latency_ms=latency * 1000,
        )
        
        # Store sample frames
        if idx % sample_interval == 0:
            overlay_frames.append(frame_overlay.copy())
            frame_indices.append(idx)
        
        process_count += 1
    
    prev_gray = curr_gray

release_capture(cap)

print(f"\n✓ Processing complete: {process_count} frames")

## Visualization: Sample Frames with Overlay

In [ ]:
# Display sample frames showing navigation state overlay
num_samples = min(6, len(overlay_frames))

if num_samples > 0:
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for i in range(num_samples):
        axes[i].imshow(overlay_frames[i])
        axes[i].set_title(f"Frame {frame_indices[i]}", fontsize=12)
        axes[i].axis("off")
    
    # Hide unused subplots
    for i in range(num_samples, 6):
        axes[i].axis("off")
    
    plt.suptitle("Navigation Overlay — Sample Frames", fontsize=16, fontweight="bold")
    plt.tight_layout()
    plt.show()
else:
    print("No sample frames available")

## Performance Metrics Summary

In [ ]:
# Print comprehensive metrics summary
metrics.print_summary()

## Detailed Analysis: State Distribution

In [ ]:
# Visualize time spent in each state
state_percentages = metrics.states.get_state_percentages()

if state_percentages:
    states = list(state_percentages.keys())
    percentages = list(state_percentages.values())
    
    # Color mapping
    color_map = {
        "CRUISE_FORWARD": "green",
        "AVOID_LEFT": "orange",
        "AVOID_RIGHT": "orange",
        "STOP_TOO_CLOSE": "red",
        "RECOVERY": "purple",
    }
    colors = [color_map.get(s, "gray") for s in states]
    
    plt.figure(figsize=(12, 6))
    bars = plt.bar(states, percentages, color=colors, alpha=0.7, edgecolor="black")
    plt.ylabel("Percentage of Time (%)", fontsize=12)
    plt.xlabel("Navigation State", fontsize=12)
    plt.title("Time Distribution Across Navigation States", fontsize=14, fontweight="bold")
    plt.xticks(rotation=45, ha="right")
    plt.grid(True, alpha=0.3, axis="y")
    
    # Add percentage labels on bars
    for bar, pct in zip(bars, percentages):
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{pct:.1f}%', ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
else:
    print("No state data available")

## Detailed Analysis: Performance Over Time

In [ ]:
# Compute rolling FPS and visualize latency distribution
frame_times = metrics.perf.frame_times
latencies_ms = np.array(metrics.perf.latencies) * 1000

# Rolling FPS
rolling_fps = []
window = 30
for i in range(window, len(frame_times)):
    elapsed = frame_times[i] - frame_times[i - window]
    fps = window / elapsed if elapsed > 0 else 0
    rolling_fps.append(fps)

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# FPS over time
axes[0].plot(range(window, len(frame_times)), rolling_fps, label="Rolling FPS (30 frames)", linewidth=2)
avg_fps = metrics.perf.get_fps()
axes[0].axhline(avg_fps, color="red", linestyle="--", linewidth=2, label=f"Average: {avg_fps:.1f} FPS")
axes[0].set_ylabel("FPS", fontsize=12)
axes[0].set_title("Frame Rate Over Time", fontsize=14, fontweight="bold")
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Latency distribution
axes[1].hist(latencies_ms, bins=50, alpha=0.7, color="blue", edgecolor="black")
latency_stats = metrics.perf.get_latency_stats()
axes[1].axvline(latency_stats["mean"], color="orange", linestyle="--", linewidth=2, label=f"Mean: {latency_stats['mean']:.1f} ms")
axes[1].axvline(latency_stats["p95"], color="red", linestyle="--", linewidth=2, label=f"P95: {latency_stats['p95']:.1f} ms")
axes[1].set_ylabel("Frequency", fontsize=12)
axes[1].set_xlabel("Frame Processing Latency (ms)", fontsize=12)
axes[1].set_title("Latency Distribution", fontsize=14, fontweight="bold")
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3, axis="y")

plt.tight_layout()
plt.show()

## Final Summary Report

In [ ]:
# Generate final report
summary = metrics.summary()

print("=" * 70)
print(" " * 15 + "VISION-BASED NAVIGATION DEMO COMPLETE")
print("=" * 70)
print()
print("System Components:")
print("  ✓ Optical Flow Perception (Mode A)")
print("  ✓ Temporal Memory with Hysteresis")
print("  ✓ Finite State Machine with Anti-Oscillation")
print("  ✓ Motion Command Controller")
print("  ✓ Real-time Performance Monitoring")
print()

perf = summary["performance"]
states = summary["states"]

print("Key Results:")
print(f"  Frames Processed:     {perf['frames_processed']}")
print(f"  Average FPS:          {perf['fps']:.2f}")
print(f"  Mean Latency:         {perf['latency_ms']['mean']:.2f} ms")
print(f"  P95 Latency:          {perf['latency_ms']['p95']:.2f} ms")
print(f"  State Transitions:    {states['state_transitions']}")
print(f"  Recovery Events:      {states['recovery_events']}")
print()

print("State Time Allocation:")
for state, pct in sorted(states["state_percentages"].items(), key=lambda x: -x[1]):
    print(f"  {state:20s}: {pct:5.1f}%")
print()

# Check if system meets basic requirements
meets_fps = perf['fps'] >= 5.0  # At least 5 FPS for demo
meets_latency = perf['latency_ms']['p95'] < 500  # P95 latency under 500ms
has_states = len(states["state_percentages"]) >= 1

print("System Validation:")
print(f"  ✓ FPS acceptable (>= 5 FPS):         {meets_fps}")
print(f"  ✓ Latency acceptable (P95 < 500ms):  {meets_latency}")
print(f"  ✓ FSM operational:                    {has_states}")
print()

if meets_fps and meets_latency and has_states:
    print("✓ " + "="*66 + " ✓")
    print("✓ " + " "*15 + "DEMO SUCCESSFUL - ALL SYSTEMS OPERATIONAL" + " "*10 + " ✓")
    print("✓ " + "="*66 + " ✓")
else:
    print("⚠ System may need tuning for optimal performance")

print()

## Conclusion

This demo successfully demonstrates:

1. **Camera-only perception** using optical flow to estimate obstacle risk and corridor openness
2. **Temporal memory** with rolling buffers and hysteresis to smooth noisy signals
3. **State machine navigation** with anti-oscillation recovery to prevent flip-flopping
4. **Real-time performance** with FPS and latency monitoring
5. **Reproducible execution** from a clean notebook restart

The system can be extended with:
- **Mode B**: YOLO-based object detection for improved obstacle classification
- **ROS2 integration**: Deploy on real robots
- **Depth sensors**: Fuse camera and depth for better distance estimation
- **Path planning**: Add waypoint following and global navigation

**Next Steps**: See `README.md` for usage instructions and configuration options.